<a href="https://colab.research.google.com/github/a-nagar/big_data/blob/main/Getting_Started_with_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing Spark and PySpark

Run the following block of code to install Spark and PySpark. It should take about a minute to run the first block.

See more here! http://spark.apache.org/docs/latest/api/python/

# 1. Installing PySpark in Google Colab

In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()

spark

Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,014 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,

In [2]:
spark

In [5]:
sc = spark.sparkContext

# 2. Reading Data

For this example, I am going to use a publicly available data set in a CSV format.

In [7]:
# Step 1: Download a book from an external source to file system
!wget https://www.gutenberg.org/cache/epub/1513/pg1513.txt

--2025-09-30 01:55:18--  https://www.gutenberg.org/cache/epub/1513/pg1513.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 169546 (166K) [text/plain]
Saving to: ‘pg1513.txt’

pg1513.txt          100%[===================>] 165.57K  1008KB/s    in 0.2s    

2025-09-30 01:55:19 (1008 KB/s) - ‘pg1513.txt’ saved [169546/169546]



In [8]:
book = sc.textFile("/content/pg1513.txt")

In [9]:
book.take(10)

['The Project Gutenberg eBook of Romeo and Juliet',
 '    ',
 'This ebook is for the use of anyone anywhere in the United States and',
 'most other parts of the world at no cost and with almost no restrictions',
 'whatsoever. You may copy it, give it away or re-use it under the terms',
 'of the Project Gutenberg License included with this ebook or online',
 'at www.gutenberg.org. If you are not located in the United States,',
 'you will have to check the laws of the country where you are located',
 'before using this eBook.',
 '']

In [10]:
words = book.flatMap(lambda x: x.split(" ")).map(lambda x: x.lower())
longWords = words.filter(lambda x: len(x) > 5)
wordPairs = longWords.map(lambda x: (x, 1))
wordCounts = wordPairs.reduceByKey(lambda x,y: x + y).sortBy(lambda x: -x[1])

In [11]:
wordCounts.take(10)

[('romeo.', 188),
 ('juliet.', 132),
 ('capulet.', 108),
 ('nurse.', 104),
 ('project', 84),
 ('mercutio.', 67),
 ('benvolio.', 65),
 ('lawrence.', 58),
 ('gutenberg™', 54),
 ('should', 50)]

In [12]:
import re

In [16]:
words = book.flatMap(lambda x: re.split(r'[^a-zA-Z]+', x)).map(lambda x: x.lower())
longWords = words.filter(lambda x: len(x) > 5)
wordPairs = longWords.map(lambda x: (x, 1))
wordCounts = wordPairs.reduceByKey(lambda x,y: x + y).sortBy(lambda x: -x[1])

In [17]:
wordCounts.take(10)

[('juliet', 194),
 ('capulet', 163),
 ('gutenberg', 98),
 ('project', 89),
 ('mercutio', 88),
 ('lawrence', 82),
 ('tybalt', 80),
 ('benvolio', 79),
 ('should', 51),
 ('montague', 47)]

#3. PySpark DataFrames

In [19]:
!wget https://raw.githubusercontent.com/a-nagar/datasets/refs/heads/main/movielens/movies.csv
!wget https://raw.githubusercontent.com/a-nagar/datasets/refs/heads/main/movielens/ratings.csv
!wget https://raw.githubusercontent.com/a-nagar/datasets/refs/heads/main/movielens/tags.csv


--2025-09-30 02:02:14--  https://raw.githubusercontent.com/a-nagar/datasets/refs/heads/main/movielens/movies.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 494431 (483K) [text/plain]
Saving to: ‘movies.csv’

movies.csv          100%[===================>] 482.84K  --.-KB/s    in 0.03s   

2025-09-30 02:02:14 (17.5 MB/s) - ‘movies.csv’ saved [494431/494431]

--2025-09-30 02:02:14--  https://raw.githubusercontent.com/a-nagar/datasets/refs/heads/main/movielens/ratings.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2483723 (2

In [22]:
moviesDF = spark.read.option("header","true").option("inferSchema","true").csv("/content/movies.csv")
ratingsDF = spark.read.option("header","true").option("inferSchema","true").csv("/content/ratings.csv")
tagsDF = spark.read.option("header","true").option("inferSchema","true").csv("/content/tags.csv")

In [23]:
moviesDF.show(10)

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
+-------+--------------------+--------------------+
only showing top 10 rows



In [25]:
ratingsDF.show(10)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
+------+-------+------+---------+
only showing top 10 rows



In [26]:
tagsDF.show(10)

+------+-------+-----------------+----------+
|userId|movieId|              tag| timestamp|
+------+-------+-----------------+----------+
|     2|  60756|            funny|1445714994|
|     2|  60756|  Highly quotable|1445714996|
|     2|  60756|     will ferrell|1445714992|
|     2|  89774|     Boxing story|1445715207|
|     2|  89774|              MMA|1445715200|
|     2|  89774|        Tom Hardy|1445715205|
|     2| 106782|            drugs|1445715054|
|     2| 106782|Leonardo DiCaprio|1445715051|
|     2| 106782|  Martin Scorsese|1445715056|
|     7|  48516|     way too long|1169687325|
+------+-------+-----------------+----------+
only showing top 10 rows



In [32]:
# Which movie has the highest count of ratings
from pyspark.sql.functions import count, desc, avg
ratingsDF.groupBy("movieId").count().orderBy(desc("count")).show()

+-------+-----+
|movieId|count|
+-------+-----+
|    356|  329|
|    318|  317|
|    296|  307|
|    593|  279|
|   2571|  278|
|    260|  251|
|    480|  238|
|    110|  237|
|    589|  224|
|    527|  220|
|   2959|  218|
|      1|  215|
|   1196|  211|
|     50|  204|
|   2858|  204|
|     47|  203|
|    780|  202|
|    150|  201|
|   1198|  200|
|   4993|  198|
+-------+-----+
only showing top 20 rows



In [28]:
# Find the avg ratings for each movie
avgRatings = ratingsDF.groupBy("movieId").avg("rating")
avgRatings.show()

+-------+------------------+
|movieId|       avg(rating)|
+-------+------------------+
|   1580| 3.487878787878788|
|   2366|              3.64|
|   3175|              3.58|
|   1088| 3.369047619047619|
|  32460|              4.25|
|  44022| 3.217391304347826|
|  96488|              4.25|
|   1238| 4.055555555555555|
|   1342|               2.5|
|   1591|2.6346153846153846|
|   1645| 3.411764705882353|
|   4519|3.3333333333333335|
|   2142|               2.7|
|    471|              3.55|
|   3997|1.8333333333333333|
|    833|               2.0|
|   3918|3.2777777777777777|
|   7982|              3.25|
|   1959|3.6666666666666665|
|  68135|              3.55|
+-------+------------------+
only showing top 20 rows



In [36]:
# Let's keep only the movies with at least 100 count and then find their average ratings
highCountRatingsAvg = ratingsDF.groupBy("movieId").agg(count("rating").alias("count"), avg("rating").alias("avg"))
highCountRatingsAvg = highCountRatingsAvg.filter("count >= 100")
# Let's join this dataframe to moviesDF to find titles of the movies as well
highCountRatingsAvg = highCountRatingsAvg.join(moviesDF, highCountRatingsAvg.movieId == moviesDF.movieId)
highCountRatingsAvg.orderBy(desc("avg")).show(truncate=False)

+-------+-----+------------------+-------+------------------------------------------------------------------------------+---------------------------------------+
|movieId|count|avg               |movieId|title                                                                         |genres                                 |
+-------+-----+------------------+-------+------------------------------------------------------------------------------+---------------------------------------+
|318    |317  |4.429022082018927 |318    |Shawshank Redemption, The (1994)                                              |Crime|Drama                            |
|858    |192  |4.2890625         |858    |Godfather, The (1972)                                                         |Crime|Drama                            |
|2959   |218  |4.272935779816514 |2959   |Fight Club (1999)                                                             |Action|Crime|Drama|Thriller            |
|1221   |129  |4.25968992248